# Create celltypes excel table from Cell Types Ontology

## Context

- See JIRA task [DKE-1041](https://bbpteam.epfl.ch/project/issues/browse/DKE-1041)
- Georges Khazen [AnnotationMappingTable](https://docs.google.com/spreadsheets/d/1Ky0FA1XaJru9od9lze9d_7ZaFSi7OkqH/edit#gid=2119141304)
- Will be implemented below for `Neuron Type`

## Imports

In [ ]:
import rdflib
import pandas as pd
import getpass
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
from kgforge.core import KnowledgeGraphForge
from kgforge.core.commons.strategies import ResolvingStrategy

# Using the celltypes.ttl

## Load Cell Types Ontology

Downloaded from WebProtégé

In [ ]:
cell_types_ontology = rdflib.Graph()
cell_types_ontology.parse("/Users/akkaufma/Desktop/celltypes.ttl") # TODO

## Load Brain Region Ontology

Downloaded from WebProtégé

In [ ]:
brainregion_ontology = rdflib.Graph()
brainregion_ontology.parse("/Users/akkaufma/Desktop/brainregion.ttl") # TODO

## Query

In [ ]:
    query = f"""

       PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
       PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
       PREFIX commonshapes: <https://neuroshapes.org/commons/>
       PREFIX datashapes: <https://neuroshapes.org/dash/>
       PREFIX dc: <http://purl.org/dc/elements/1.1/>
       PREFIX dcat: <http://www.w3.org/ns/dcat#>
       PREFIX dcterms: <http://purl.org/dc/terms/>
       PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
       PREFIX nsg: <https://neuroshapes.org/>
       PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
       PREFIX oa: <http://www.w3.org/ns/oa#>
       PREFIX obo: <http://purl.obolibrary.org/obo/>
       PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX prov: <http://www.w3.org/ns/prov#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX schema: <http://schema.org/>
       PREFIX sh: <http://www.w3.org/ns/shacl#>
       PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
       PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
       PREFIX vann: <http://purl.org/vocab/vann/>
       PREFIX void: <http://rdfs.org/ns/void#>
       PREFIX xml: <http://www.w3.org/XML/1998/namespace/>
       PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   
   
       SELECT 
           ?brain_region_label 
           ?brain_region_id 
           ?species_label
           ?species_id
           ?m_type_label
           ?m_type_id
           ?e_type_label
           ?e_type_id
           ?t_type_label
           ?t_type_id
           ?t_type_source
           ?transmitter_type_label
           ?transmitter_type_id
   
       WHERE {{
            ?m_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronMorphologicalType> ;
                       rdfs:label ?m_type_label .
               
               
            OPTIONAL {{ 
               ?m_type_id rdfs:subClassOf* ?region_restriction .
               ?region_restriction a owl:Restriction ;
                   owl:onProperty bmo:canBeLocatedInBrainRegion ;
                   owl:someValuesFrom ?brain_region_id .
                }} .
                   
            OPTIONAL {{
               ?m_type_id rdfs:subClassOf* ?transmitter_restriction .
               ?transmitter_restriction a owl:Restriction ;
                   owl:onProperty <https://bbp.epfl.ch/ontologies/core/mtypes/hasNeurotransmitterType> ;
                   owl:someValuesFrom ?transmitter_type_id .
               ?transmitter_type_id rdfs:label ?transmitter_type_label .
               }} .
               
            OPTIONAL {{
            
                ?m_type_id rdfs:subClassOf* ?species_restriction .
                ?species_restriction a owl:Restriction ;
                   owl:onProperty <https://neuroshapes.org/hasInstanceInSpecies> ;
                   owl:someValuesFrom ?species_id .
                ?species_id rdfs:label ?species_label .
            }} .
            
            OPTIONAL {{
            
                ?e_type_id rdfs:subClassOf* ?mtype_restriction .
                ?mtype_restriction a owl:Restriction ;
                   owl:onProperty <https://bbp.epfl.ch/ontologies/core/bmo/canHaveMType> ;
                   owl:someValuesFrom ?m_type_id .
                ?e_type_id rdfs:label ?e_type_label .
            }} .
            
            OPTIONAL {{
            
                ?m_type_id rdfs:subClassOf* ?ttype_restriction .
                ?ttype_restriction a owl:Restriction ;
                   owl:onProperty <https://bbp.epfl.ch/ontologies/core/bmo/canHaveTType> ;
                   owl:someValuesFrom ?t_type_id .
                ?t_type_id rdfs:label ?t_type_label .
                
                OPTIONAL {{
                    ?m_type_id rdfs:seeAlso ?cell_type_source 
                }} .
                
            }} .
            
            FILTER NOT EXISTS {{ ?s rdfs:subClassOf ?m_type_id }} .

       }}
         LIMIT 1000
    """

In [ ]:
rows = list()
query = query
result = cell_types_ontology.query(query)
for row in result:
    rows.append(row)

## Save to excel

In [ ]:
df = pd.DataFrame(rows, columns=["brain_region_label", 
                             "brain_region_id", 
                             "species_label",
                             "species_id",
                             "m_type_label",
                             "m_type_id",
                             "e_type_label",
                             "e_type_id",
                             "t_type_label",
                             "t_type_id",
                             "t_type_source",
                             "transmitter_type_label",
                             "transmitter_type_id",
                             
                             ]) 

In [ ]:
df.head()

In [ ]:
for row in df.iterrows():
    br_id = str(row[1].brain_region_id)
    if br_id:
        for s, p, o in brainregion_ontology.triples((rdflib.term.URIRef(br_id), RDFS.label, None)):
            row[1].brain_region_label = o

In [ ]:
for row in df.iterrows():
    m_id = str(row[1].m_type_id)
    if m_id:
        for s, p, o in cell_types_ontology.triples((rdflib.term.URIRef(m_id), RDFS.label, None)):
            row[1].m_type_label = o

In [ ]:
df.to_excel("./celltypes.xlsx")

# Using Nexus

## Setup

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="neurosciencegraph/datamodels",
                            token=TOKEN)

## Query

In [ ]:
query = f"""
        
       PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
       PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
       PREFIX commonshapes: <https://neuroshapes.org/commons/>
       PREFIX datashapes: <https://neuroshapes.org/dash/>
       PREFIX dc: <http://purl.org/dc/elements/1.1/>
       PREFIX dcat: <http://www.w3.org/ns/dcat#>
       PREFIX dcterms: <http://purl.org/dc/terms/>
       PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
       PREFIX nsg: <https://neuroshapes.org/>
       PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
       PREFIX oa: <http://www.w3.org/ns/oa#>
       PREFIX obo: <http://purl.obolibrary.org/obo/>
       PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX prov: <http://www.w3.org/ns/prov#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX schema: <http://schema.org/>
       PREFIX sh: <http://www.w3.org/ns/shacl#>
       PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
       PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
       PREFIX vann: <http://purl.org/vocab/vann/>
       PREFIX void: <http://rdfs.org/ns/void#>
       PREFIX xml: <http://www.w3.org/XML/1998/namespace/>
       PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
       
       SELECT DISTINCT
           ?brain_region_label 
           ?brain_region_id 
           ?species_label
           ?species_id
           ?m_type_label
           ?m_type_id
           ?e_type_label
           ?e_type_id
           ?t_type_label
           ?t_type_id
           ?t_type_source
           ?transmitter_type_label
           ?transmitter_type_id
   
       WHERE {{
               
               ?m_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronMorphologicalType> ;
               rdfs:label ?m_type_label ;
               <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
               <https://bluebrain.github.io/nexus/vocabulary/updatedAt> ?date .
               
               FILTER (?date > "2022-10-29T00:00:00+00:00"^^xsd:dateTime)
                              
               OPTIONAL {{ ?m_type_id rdfs:subClassOf* / bmo:canBeLocatedInBrainRegion ?brain_region_id .
               ?brain_region_id rdfs:label ?brain_region_label .
               }}
                   
               OPTIONAL {{ ?m_type_id rdfs:subClassOf* / <https://bbp.epfl.ch/ontologies/core/mtypes/hasNeurotransmitterType> ?transmitter_type_id .
               ?transmitter_type_id rdfs:label ?transmitter_type_label .
               }}
            
               OPTIONAL {{ ?e_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronElectricalType> ;
                   <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
                   bmo:canHaveMType ?m_type_id ;
                   rdfs:label ?e_type_label .            
               }}
               
               OPTIONAL {{ ?m_type_id bmo:canHaveTType ?t_type_id .
                   ?t_type_id rdfs:label ?t_type_label .
                   
                   OPTIONAL {{ ?e_type_id bmo:canHaveTType ?t_type_id ;
                       rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronElectricalType> ;
                       <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
                       bmo:canHaveMType ?m_type_id ;
                       rdfs:label ?e_type_label .
                    }}
                   OPTIONAL {{
                       ?t_type_id rdfs:seeAlso ?t_type_source .
                   }} .
               }}
               
               FILTER NOT EXISTS {{ ?s rdfs:subClassOf ?m_type_id }} .
               
       }}

    """

In [ ]:
resources = forge.sparql(query, limit=100000, rewrite=False, debug=False)

In [ ]:
len(resources)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
df.to_excel("/Users/akkaufma/Desktop/celltypes_nexus.xlsx")

## Supplemental e-types

In [ ]:
query = f"""
        
       PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
       PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
       PREFIX commonshapes: <https://neuroshapes.org/commons/>
       PREFIX datashapes: <https://neuroshapes.org/dash/>
       PREFIX dc: <http://purl.org/dc/elements/1.1/>
       PREFIX dcat: <http://www.w3.org/ns/dcat#>
       PREFIX dcterms: <http://purl.org/dc/terms/>
       PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
       PREFIX nsg: <https://neuroshapes.org/>
       PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
       PREFIX oa: <http://www.w3.org/ns/oa#>
       PREFIX obo: <http://purl.obolibrary.org/obo/>
       PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX prov: <http://www.w3.org/ns/prov#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX schema: <http://schema.org/>
       PREFIX sh: <http://www.w3.org/ns/shacl#>
       PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
       PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
       PREFIX vann: <http://purl.org/vocab/vann/>
       PREFIX void: <http://rdfs.org/ns/void#>
       PREFIX xml: <http://www.w3.org/XML/1998/namespace/>
       PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
       
       SELECT DISTINCT
           ?brain_region_label 
           ?brain_region_id 
           ?species_label
           ?species_id
           ?m_type_label
           ?m_type_id
           ?e_type_label
           ?e_type_id
           ?t_type_label
           ?t_type_id
           ?t_type_source
           ?transmitter_type_label
           ?transmitter_type_id
   
       WHERE {{

               ?e_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronElectricalType> ;
                   <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
                   rdfs:label ?e_type_label .   
               
               OPTIONAL {{ ?e_type_id rdfs:subClassOf* / bmo:canBeLocatedInBrainRegion ?brain_region_id .
               ?brain_region_id rdfs:label ?brain_region_label .
               }}
                             
               FILTER NOT EXISTS {{ ?s rdfs:subClassOf ?e_type_id }} .
               
               ?e_type_id <https://bluebrain.github.io/nexus/vocabulary/updatedAt> ?date .
               FILTER (?date > "2022-10-29T00:00:00+00:00"^^xsd:dateTime)
               
       }}

    """

In [ ]:
etypes = forge.sparql(query, limit=100000, rewrite=False, debug=False)

In [ ]:
df_2 = forge.as_dataframe(etypes)

In [ ]:
df_2

In [ ]:
for row in df_2.iterrows():
    if row[1].e_type_id not in df.e_type_id.to_list():
        e_type_id = el
        e_type_label = forge.retrieve(el).label
        df = df.append(df_2.iloc[row[0]])

In [ ]:
df = df.drop(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
df

In [ ]:
df.to_excel("./celltypes_nexus.xlsx")

## Without t-types

In [ ]:
query = f"""
        
       PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
       PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
       PREFIX commonshapes: <https://neuroshapes.org/commons/>
       PREFIX datashapes: <https://neuroshapes.org/dash/>
       PREFIX dc: <http://purl.org/dc/elements/1.1/>
       PREFIX dcat: <http://www.w3.org/ns/dcat#>
       PREFIX dcterms: <http://purl.org/dc/terms/>
       PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
       PREFIX nsg: <https://neuroshapes.org/>
       PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
       PREFIX oa: <http://www.w3.org/ns/oa#>
       PREFIX obo: <http://purl.obolibrary.org/obo/>
       PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX prov: <http://www.w3.org/ns/prov#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX schema: <http://schema.org/>
       PREFIX sh: <http://www.w3.org/ns/shacl#>
       PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
       PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
       PREFIX vann: <http://purl.org/vocab/vann/>
       PREFIX void: <http://rdfs.org/ns/void#>
       PREFIX xml: <http://www.w3.org/XML/1998/namespace/>
       PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
       
       SELECT DISTINCT
           ?brain_region_label 
           ?brain_region_id 
           ?species_label
           ?species_id
           ?m_type_label
           ?m_type_id
           ?e_type_label
           ?e_type_id
           ?transmitter_type_label
           ?transmitter_type_id
   
       WHERE {{
               
               ?m_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronMorphologicalType> ;
               rdfs:label ?m_type_label ;
               <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
               <https://bluebrain.github.io/nexus/vocabulary/updatedAt> ?date .
               
               FILTER (?date > "2022-10-29T00:00:00+00:00"^^xsd:dateTime)
                              
               OPTIONAL {{ ?m_type_id rdfs:subClassOf* / bmo:canBeLocatedInBrainRegion ?brain_region_id .
               ?brain_region_id rdfs:label ?brain_region_label .
               }}
                   
               OPTIONAL {{ ?m_type_id rdfs:subClassOf* / <https://bbp.epfl.ch/ontologies/core/mtypes/hasNeurotransmitterType> ?transmitter_type_id .
               ?transmitter_type_id rdfs:label ?transmitter_type_label .
               }}
            
               OPTIONAL {{ ?e_type_id rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/bmo/NeuronElectricalType> ;
                   <https://bluebrain.github.io/nexus/vocabulary/deprecated> False ;
                   bmo:canHaveMType ?m_type_id ;
                   rdfs:label ?e_type_label .            
               }}
               
               FILTER NOT EXISTS {{ ?s rdfs:subClassOf ?m_type_id }} .
               
       }}

    """

In [ ]:
resources = forge.sparql(query, limit=100000, rewrite=False, debug=False)

In [ ]:
len(resources)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.to_excel("/Users/akkaufma/Desktop/celltypes_nexus_2.xlsx")